In [34]:
import sys
import math
import numpy as np
import pdb
import matplotlib.pyplot as plt

In [35]:
beamx_c = -2.4
beamy_c = 0.0
beamxi_c = -9.0
x_c = -2.4
y_c = 0.0
xi_c = -9.0
s1 = 1
s2 = 6
xdensity = 20
ydensity = 8
xidensity = 30
sigma_x = 0.1
sigma_y = 0.2
xidensity_ = xidensity + xdensity - 1
t0=0
noObj= ydensity * xidensity_

In [36]:
useWeights_x = True
useWeights_y = True
useMasks_xi = True
useMasks_y = True

In [37]:
# Recompute necessary parameters (as done in shape file)
xidensity_ = xidensity + xdensity - 1  # Allows enough particles in xi direction for x layering
xistep = 2*s2/xidensity # Can also use resolution here
ystep = 2*s1/ydensity
xstep = xistep          # Purposefully setting xstep as equal to xistep for projection of x onto xi, unused here

# Calculate s3 
s3 = xstep*(xdensity-1)/2.0
print(f"s3 = {s3}")

# Define corners (front is first to enter field)
ytop = y_c + s1
ybot = y_c - s1
xileft = xi_c - s2
xiright = xi_c + s2
xfront = x_c + s3  
xback = x_c - s3

# Start in front top right
yn = ytop
xin = xiright
xn = xfront
zn = xiright + t0

s3 = 3.8000000000000003


In [38]:
# Create individual coordiante arrays    
x_0 = np.linspace(xfront,xback,xdensity)
y_0 = np.linspace(ytop,ybot,ydensity)
xi_0 = np.linspace(xiright,xileft,xidensity)
    

In [39]:
%%timeit
w=[]
w = [0 for k in range(0,noObj)]
for i in range(0,len(x_0)):
    # Create 3d virtual coordinate array for x-slice
    xv, yv, xiv = np.meshgrid(x_0[i], y_0, xi_0,indexing='ij',sparse=True)
    
    # Create 3d virtual weight arrays containing weight of each particle in x-slice
    #w_x = np.exp((-1.*(xv-beamx_c)**2)/(2*sigma_x**2))
    w_x = np.exp((-1.*(x_0[i]-beamx_c)**2)/(2*sigma_x**2))
    w_y = np.exp((-1.*(yv-beamy_c)**2)/(2*sigma_y**2))
    
    # Weighting options evaluator
    if (useWeights_x) and (useWeights_y):
        w_virt = w_x * w_y
    elif (useWeights_x):
        w_virt = w_x
    elif (useWeights_y):
        w_virt = w_y

    # MASKING 
    if (useMasks_xi):
        #Define masks in z direction, leftmost z-coordinate = 0. Change if different mask is desired
        left_of_masks= [-12]  # left most limit of each mask in order
        right_of_masks = [-9]  # right most limit of each mask in order

        # Conversion of z-coord to xi-coord
        for m in range(0,len(left_of_masks)):
            left_of_masks[m] = left_of_masks[m] - zn
            right_of_masks[m] = right_of_masks[m] - zn

        for g in range(0,len(left_of_masks)):
            w_virt = np.where(np.logical_and(xiv > left_of_masks[g], xiv < right_of_masks[g]), 0, w_virt)
            
    if (useMasks_y):
        #Define masks in y direction, 0 is 0 on the y-axis. Change if different mask is desired
        top_of_masks = [-0.3,0.8]  #upper limit of each mask in order
        bot_of_masks = [-0.4,0.5]  #lower limit of each mask in order 

        for h in range(0,len(top_of_masks)):
            w_virt = np.where(np.logical_and(yv > bot_of_masks[h], yv < top_of_masks[h]), 0, w_virt)
    # END OF MASKING    
        
    # Create final weighting list to return
    # Maps 3d virtual particles onto 2d projection
    for j in range(0,ydensity):
        for k in range(0,xidensity):
            w[xidensity_ * j + k + i] += w_virt[0,j,k]
    
    xv = None
    yv = None
    xiv = None
    w_x = None
    w_y = None
    w_virt = None

3.42 ms ± 132 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [40]:
print(w)

[1.02494e-319, 3.5408573938415096e-257, 1.3765821054467318e-201, 6.022601013723181e-153, 2.965202486435833e-111, 1.6429067270605202e-76, 1.0243768369091207e-48, 7.1877817390610525e-28, 5.675685232632805e-14, 5.043477193247397e-07, 1.0086953818926315e-06, 1.0086954386494838e-06, 1.0086954386494838e-06, 1.0086954386494838e-06, 1.0086954386494838e-06, 1.0086954386494838e-06, 1.0086953818926315e-06, 5.043477193247441e-07, 5.675685232632805e-14, 7.187781739060746e-28, 1.0243768369091207e-48, 1.0243768369091207e-48, 7.1877817390610525e-28, 5.675685232632805e-14, 5.043477193247397e-07, 1.0086953818926315e-06, 1.0086954386494838e-06, 1.0086954386494838e-06, 1.0086954386494838e-06, 1.0086954386494838e-06, 1.0086954386494838e-06, 1.0086954386494838e-06, 1.0086954386494838e-06, 1.0086954386494838e-06, 1.0086954386494838e-06, 1.0086954386494838e-06, 1.0086954386494838e-06, 1.0086954386494838e-06, 1.0086953818926315e-06, 5.043477193247441e-07, 5.675685232632805e-14, 7.187781739060746e-28, 1.0243768

In [41]:
w_false= np.load("sparse-false.npy")

In [42]:
w_false == w

<ipython-input-42-559ac11f5e8f>:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  w_false == w


False

In [43]:
np.array_equal(w_false,w)

False